In [77]:
import pandas as pd
import math

In [62]:
df = pd.read_csv('../data/feedback_goemotion.csv')

In [63]:
df.head()

,Unnamed: 0,text,labels,scores,better label(s) - Justin,Other - Justin
0,0,u still there,['neutral'],[0.9985159],NaN,NaN
1,1,call my #,['neutral'],[0.99839693],NaN,NaN
2,2,"Really, you are better than this (calling thes...",['admiration' 'approval'],[0.9710561 0.9628638],"admiration, caring",NaN
3,3,We're a bunch of guys working to reduce demand...,['curiosity'],[0.98822975],NaN,NaN
4,4,"lots of danger, including jail",['fear' 'neutral'],[0.89338058 0.60538208],NaN,Accountability


In [64]:
df.drop(labels='Unnamed: 0', axis=1, inplace=True)

In [65]:
df.head()

,text,labels,scores,better label(s) - Justin,Other - Justin
0,u still there,['neutral'],[0.9985159],NaN,NaN
1,call my #,['neutral'],[0.99839693],NaN,NaN
2,"Really, you are better than this (calling thes...",['admiration' 'approval'],[0.9710561 0.9628638],"admiration, caring",NaN
3,We're a bunch of guys working to reduce demand...,['curiosity'],[0.98822975],NaN,NaN
4,"lots of danger, including jail",['fear' 'neutral'],[0.89338058 0.60538208],NaN,Accountability


In [66]:
df.dtypes

text                        object
labels                      object
scores                      object
better label(s) - Justin    object
Other - Justin              object
dtype: object

In [67]:
df_complete = df.iloc[:497, ]

In [68]:
df_complete.iloc[-1, ]

text                        Your # is now recorded attempting to buy sex o...
labels                                                            ['neutral']
scores                                                           [0.99679846]
better label(s) - Justin                                                  NaN
Other - Justin                                                 accountability
Name: 496, dtype: object

In [69]:
df_complete.shape

(497, 5)

In [70]:
df_complete.head()

,text,labels,scores,better label(s) - Justin,Other - Justin
0,u still there,['neutral'],[0.9985159],NaN,NaN
1,call my #,['neutral'],[0.99839693],NaN,NaN
2,"Really, you are better than this (calling thes...",['admiration' 'approval'],[0.9710561 0.9628638],"admiration, caring",NaN
3,We're a bunch of guys working to reduce demand...,['curiosity'],[0.98822975],NaN,NaN
4,"lots of danger, including jail",['fear' 'neutral'],[0.89338058 0.60538208],NaN,Accountability


In [71]:
# Accuracy of goemotion bert model
df_complete['better label(s) - Justin'].isna().sum() / len(df_complete)

0.7867203219315896

In [72]:
df_complete[df_complete['better label(s) - Justin'] == 'admiration, caring']

,text,labels,scores,better label(s) - Justin,Other - Justin
2,"Really, you are better than this (calling thes...",['admiration' 'approval'],[0.9710561 0.9628638],"admiration, caring",NaN


In [73]:
df_complete[df_complete['better label(s) - Justin'] == 'curiosity']

,text,labels,scores,better label(s) - Justin,Other - Justin
8,"sorry, i was unavailable, how can i helpu",['remorse' 'sadness'],[0.90214115 0.31946027],curiosity,NaN
31,"pics are on the ad, new pics are only for peop...",['remorse' 'neutral'],[0.38172522 0.74038649],curiosity,NaN
37,Why go there?,['neutral'],[0.99532521],curiosity,NaN
49,do you understand?,['confusion' 'curiosity'],[0.59036869 0.5860163 ],curiosity,NaN
91,Yeah been doing this for awhile. Did you know ...,['approval' 'curiosity'],[0.63890344 0.95203876],curiosity,NaN
108,Does any of this bother you as much as it does...,['curiosity' 'neutral'],[0.3464857 0.70851058],curiosity,and accountability
190,do you believe in someone/something?,['confusion' 'curiosity' 'neutral'],[0.45108095 0.75383407 0.48508221],curiosity,NaN
245,it's pretty good. how about you?,['admiration' 'curiosity'],[0.92426848 0.72966903],curiosity,NaN
268,Were you aware of that?,['curiosity' 'realization'],[0.73868245 0.53019536],curiosity,NaN
411,Can I hear a little about you first?,['neutral'],[0.99110526],curiosity,NaN


In [133]:
#test = df_complete[:10]

emotions = ['admiration', 'amusement', 'anger', 'annoyance', 'approval', 'caring', 'confusion', 'curiosity', 'desire', 'disappointment', 'disapproval', 'disgust', 'embarrassment', 'excitement', 'fear', 'gratitude', 'grief', 'joy', 'love', 'nervousness', 'optimism', 'pride', 'realization', 'relief', 'remorse', 'sadness', 'surprise', 'neutral']

for emotion in emotions:
    df_complete[emotion] = 0

for index, row in df_complete.iterrows():
    #print(row['better label(s) - Justin'])
    hand_labeled = row['better label(s) - Justin']
    if isinstance(hand_labeled, str) != True and math.isnan(hand_labeled):
        model_labeled = row['labels']
        model_labeled = model_labeled.strip("[]")
        model_labeled = model_labeled.replace("'", "")
        model_labeled = model_labeled.split()
        #print(model_labeled.split())
        for model_label in model_labeled:
            df_complete.at[index, model_label] = 1
    else:
        hand_labeled = hand_labeled.replace(" ", "")
        hand_labeled = hand_labeled.split(',')
        for hand_label in hand_labeled:
            df_complete.at[index, hand_label] = 1

/Users/mani/.local/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [135]:
df_complete[df_complete['admiration'] == 1]

,text,labels,scores,better label(s) - Justin,Other - Justin,admiration,amusement,anger,annoyance,approval,...,love,nervousness,optimism,pride,realization,relief,remorse,sadness,surprise,neutral
2,"Really, you are better than this (calling thes...",['admiration' 'approval'],[0.9710561 0.9628638],"admiration, caring",NaN,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
54,Amazing! Please tell me just one girls story,['admiration'],[0.90544182],"curiosity, admiration",NaN,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
74,oh yeah u like that? hehe ur cute,['admiration' 'amusement'],[0.49104971 0.95805871],NaN,Flirtatious,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
124,Wow u r brave...lots of scary girls on here. L...,['admiration' 'surprise'],[0.88604105 0.95555055],NaN,sarcastic,1,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
138,You are better than this!,['admiration' 'approval'],[0.62984532 0.98096037],NaN,NaN,1,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
149,awesome. are you aware that buying sex is ille...,['admiration' 'curiosity'],[0.97230703 0.34766856],NaN,NaN,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
155,"Good stuff. Anyway, I have to go. It was goo...",['admiration' 'optimism'],[0.97715604 0.94778788],NaN,NaN,1,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
286,"hey kevin, im doin great. whatcha up to",['admiration'],[0.97747886],NaN,NaN,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
322,"so so good, what about you chief? :)",['admiration' 'curiosity'],[0.92128998 0.95533746],NaN,NaN,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
342,Really good! How can I help???,['admiration' 'curiosity'],[0.9890542 0.85639095],NaN,NaN,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [136]:
df_complete.head()

,text,labels,scores,better label(s) - Justin,Other - Justin,admiration,amusement,anger,annoyance,approval,...,love,nervousness,optimism,pride,realization,relief,remorse,sadness,surprise,neutral
0,u still there,['neutral'],[0.9985159],NaN,NaN,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,call my #,['neutral'],[0.99839693],NaN,NaN,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,"Really, you are better than this (calling thes...",['admiration' 'approval'],[0.9710561 0.9628638],"admiration, caring",NaN,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,We're a bunch of guys working to reduce demand...,['curiosity'],[0.98822975],NaN,NaN,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"lots of danger, including jail",['fear' 'neutral'],[0.89338058 0.60538208],NaN,Accountability,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [138]:
df_complete.drop(labels=['labels', 'scores', 'better label(s) - Justin', 'Other - Justin'], axis=1, inplace=True)

/Users/mani/Desktop/epik-project-nlp/venv/lib/python3.7/site-packages/pandas/core/frame.py:4312: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [139]:
df_complete.head()

,text,admiration,amusement,anger,annoyance,approval,caring,confusion,curiosity,desire,...,love,nervousness,optimism,pride,realization,relief,remorse,sadness,surprise,neutral
0,u still there,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,call my #,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,"Really, you are better than this (calling thes...",1,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,We're a bunch of guys working to reduce demand...,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
4,"lots of danger, including jail",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [141]:
df_complete.to_csv('../data/labeled_data_1000.csv')